In [15]:
directions = {
    "^": (0, -1),
    "v": (0, 1),
    "<": (-1, 0),
    ">": (1, 0)
}

In [16]:
class Box:
    def __init__(self,x,y, zone):
        self.x = x
        self.y = y
        self.zone = zone

    @property
    def position(self):
        return (self.x,self.y)
    
    @property
    def gps(self):
        return 100*self.y + self.x
    
    def move(self, x, y):
        next_pos = self.zone[self.y + y][self.x + x]
        if next_pos == ".":
            self.x += x
            self.y += y
            self.zone[self.y][self.x] = self
            return True
        if next_pos == "#":
            return False
        if isinstance(next_pos, Box):
            if not next_pos.move(x , y):
                return False
            self.x += x
            self.y += y
            self.zone[self.y][self.x] = self
            return True
        
    def __repr__(self):
        return " O "


In [17]:
class Box2:
    def __init__(self,x,y, zone):
        self.x1 = x
        self.x2 = x + 1
        self.y = y
        self.zone = zone

    @property
    def position(self):
        return (self.x1,self.y)
    
    @property
    def gps(self):
        return 100*self.y + self.x1
    
    def check_move(self, x, y):
        x1 = x
        if x != 0:
            if x == 1:
                x1 = 2
            elif x == -1:
                x1 = -1
        next_pos1 = self.zone[self.y + y][self.x1 + x1]

        if y != 0:
            next_pos2 = self.zone[self.y + y][self.x2]
            if next_pos1 == "." and next_pos2 == ".":
                return True
            if next_pos1 == "#" or next_pos2 == "#":
                return False
            
            side1 = True
            side2 = True

            if isinstance(next_pos1, Box2):
                side1 = next_pos1.check_move(0, y)
            if isinstance(next_pos2, Box2) and (next_pos2 != next_pos1):
                side2 = next_pos2.check_move(0, y)
            return side1 and side2
        else:
            if next_pos1 == ".":
                return True
            if next_pos1 == "#":
                return False
            if isinstance(next_pos1, Box2):
                return next_pos1.check_move(x, 0)
            return True
            

    def move(self, x, y):
        x1 = x
        if x != 0:
            if x == 1:
                x1 = 2
            elif x == -1:
                x1 = -1
        next_pos1 = self.zone[self.y + y][self.x1 + x1]
        
        if y != 0:
            next_pos2 = self.zone[self.y + y][self.x2]
            if isinstance(next_pos1, Box2):
                next_pos1.move(0, y)
            if isinstance(next_pos2, Box2) and (next_pos2 != next_pos1):
                next_pos2.move(0, y)
            self.zone[self.y][self.x1] = "."
            self.zone[self.y][self.x2] = "."
            self.y += y
            self.zone[self.y][self.x1] = self
            self.zone[self.y][self.x2] = self
            return True
        else:
            if isinstance(next_pos1, Box2):
                next_pos1.move(x, 0)
            self.zone[self.y][self.x1] = "."
            self.zone[self.y][self.x2] = "."
            self.x1 += x
            self.x2 += x
            self.zone[self.y][self.x1] = self
            self.zone[self.y][self.x2] = self
            return True
                
    def __repr__(self):
        return "O"

In [18]:
def open_file(file):
    zone = []
    boxes = []
    initial_pos = None
    moves = []
    with open(file) as f:
        z, m = f.read().split('\n\n')
        
        for y, line in enumerate(z.split('\n')):
            zone.append([])
            for x, c in enumerate(list(line)):
                if c == "#":
                    zone[y].append("#")
                elif c == ".":
                    zone[y].append(".")
                elif c == "O":
                    box = Box(x, y, zone)
                    zone[y].append(box)
                    boxes.append(box)
                elif c == "@":
                    initial_pos = (x, y)
                    zone[y].append(".")

        m = map(list,m.split('\n'))
        moves = [element for sublist in m for element in sublist]
        moves = list(map(lambda x: directions[x], moves))

    return zone, boxes, initial_pos, moves

In [19]:
def open_file2(file):
    zone = []
    boxes = []
    initial_pos = None
    moves = []
    with open(file) as f:
        z, m = f.read().split('\n\n')
        
        for y, line in enumerate(z.split('\n')):
            zone.append([])
            for x, c in enumerate(list(line)):
                if c == "#":
                    zone[y].extend(["#", "#"])
                elif c == ".":
                    zone[y].extend([".", "."])
                elif c == "O":
                    box2 = Box2(x*2, y, zone)
                    zone[y].extend([box2, box2])
                    boxes.append(box2)
                elif c == "@":
                    initial_pos = (x*2, y)
                    zone[y].extend(["@", "."])

        m = map(list,m.split('\n'))
        moves = [element for sublist in m for element in sublist]
        moves = list(map(lambda x: directions[x], moves))
    
    return zone, boxes, initial_pos, moves

# Example

In [20]:
initial_pos = None
zone = []
moves = []
boxes = []

zone, boxes, initial_pos, moves = open_file("example_day15.txt")

zone

[['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '.', '.',  O , '.', '.',  O , '.',  O , '#'],
 ['#', '.', '.', '.', '.', '.', '.',  O , '.', '#'],
 ['#', '.',  O ,  O , '.', '.',  O , '.',  O , '#'],
 ['#', '.', '.',  O , '.', '.', '.',  O , '.', '#'],
 ['#',  O , '#', '.', '.',  O , '.', '.', '.', '#'],
 ['#',  O , '.', '.',  O , '.', '.',  O , '.', '#'],
 ['#', '.',  O ,  O , '.',  O , '.',  O ,  O , '#'],
 ['#', '.', '.', '.', '.',  O , '.', '.', '.', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']]

In [21]:
current = [initial_pos[0], initial_pos[1]]

for move in moves:
    x,y = move
    next_pos = zone[current[1] + y][current[0] + x]

    if next_pos == ".": 
        current[0] += x
        current[1] += y
    elif next_pos == "#":
        continue
    elif isinstance(next_pos, Box):
        if not next_pos.move(x , y):
            continue
        current[0] += x
        current[1] += y
        zone[current[1]][current[0]] = "."

zone

[['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '.',  O , '.',  O , '.',  O ,  O ,  O , '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#',  O ,  O , '.', '.', '.', '.', '.', '.', '#'],
 ['#',  O ,  O , '.', '.', '.', '.', '.', '.', '#'],
 ['#',  O , '#', '.', '.', '.', '.', '.',  O , '#'],
 ['#',  O , '.', '.', '.', '.', '.',  O ,  O , '#'],
 ['#',  O , '.', '.', '.', '.', '.',  O ,  O , '#'],
 ['#',  O ,  O , '.', '.', '.', '.',  O ,  O , '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']]

In [22]:
gps = sum([box.gps for box in boxes])
gps

10092

In [23]:
initial_pos = None
zone = []
moves = []
boxes = []

zone, boxes, initial_pos, moves = open_file2("example_day15.txt")
for line in zone:
    print("".join(map(str,line)))

####################
##....OO....OO..OO##
##............OO..##
##..OOOO....OO..OO##
##....OO@.....OO..##
##OO##....OO......##
##OO....OO....OO..##
##..OOOO..OO..OOOO##
##........OO......##
####################


In [24]:
current = [initial_pos[0], initial_pos[1]]

for move in moves:
    x,y = move
    next_pos = zone[current[1] + y][current[0] + x]

    if next_pos == ".": 
        zone[current[1]][current[0]] = "."
        current[0] += x
        current[1] += y
        zone[current[1]][current[0]] = "@"
    elif next_pos == "#":
        continue
    elif isinstance(next_pos, Box2):
        if not next_pos.check_move(x , y):
            continue
        next_pos.move(x, y)
        zone[current[1]][current[0]] = "."
        current[0] += x
        current[1] += y
        zone[current[1]][current[0]] = "@"

for line in zone:
    print("".join(map(str,line)))

####################
##OO.......OO.OOOO##
##OO...........OO.##
##OO........OOOOOO##
##OO......OO....OO##
##..##......OO....##
##..OO............##
##..@......OO.OOOO##
##......OOOO..OO..##
####################


In [25]:
gps = sum([box.gps for box in boxes])
gps

9021

# Part 1

In [26]:
initial_pos = None
zone = []
moves = []
boxes = []

zone, boxes, initial_pos, moves = open_file("data_day15.txt")

In [27]:
current = [initial_pos[0], initial_pos[1]]

for move in moves:
    x,y = move
    next_pos = zone[current[1] + y][current[0] + x]

    if next_pos == ".": 
        current[0] += x
        current[1] += y
    elif next_pos == "#":
        continue
    elif isinstance(next_pos, Box):
        if not next_pos.move(x , y):
            continue
        current[0] += x
        current[1] += y
        zone[current[1]][current[0]] = "."

In [28]:
gps = sum([box.gps for box in boxes])
gps

1421727

# Part 2

In [29]:
initial_pos = None
zone = []
moves = []
boxes = []

zone, boxes, initial_pos, moves = open_file2("data_day15.txt")

In [30]:
current = [initial_pos[0], initial_pos[1]]

for move in moves:
    x,y = move
    next_pos = zone[current[1] + y][current[0] + x]

    if next_pos == ".": 
        zone[current[1]][current[0]] = "."
        current[0] += x
        current[1] += y
        zone[current[1]][current[0]] = "@"
    elif next_pos == "#":
        continue
    elif isinstance(next_pos, Box2):
        if not next_pos.check_move(x , y):
            continue
        next_pos.move(x, y)
        zone[current[1]][current[0]] = "."
        current[0] += x
        current[1] += y
        zone[current[1]][current[0]] = "@"

for line in zone:
    print("".join(map(str,line)))

####################################################################################################
##......OOOO......##......OOOO..OO..OO..##OOOO..........OOOO..OOOOOOOOOO##OO..........OO..OO..OOOO##
##..##..OOOO....##........OO........##........................OO...........OO.OOOO...OO.##....OO..##
######.OO.OO..@...............................................OO####........##OO......OOOOOO....OO##
##OOOOOO..OO......OO..OO.................OO................................OOOO.......OOOO####....##
##OOOOOO.....................OO.........######OO..............................OO........##..OOOO..##
##..##OOOO......##........................OOOOOO..............................OOOO##...OO..OOOOOO.##
##..OO##..........OO..........................####OO............OO....##..OOOOOO........OO....OO..##
##...........OO...OO..........OO##...............OOOOOOOO..............OO.##OO.OO.......OO...OO...##
##..........OOOO........OO......##.........OOOO...##..##...OOOO...........OO.OO.........OO.

In [31]:
gps = sum([box.gps for box in boxes])
gps

1463160